In [0]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import cv2
import numpy as np
import os
from PIL import Image

data_folder = "/content/drive/My Drive/NN-ProjectC/Project_C1/Training/augmented_images"
dataset_train = np.ndarray(shape=(1868, 3, 224, 224),
                     dtype=np.float32)

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

# dataset_train = []
i = 0
for folder in os.listdir(data_folder):
    sub_folder = os.path.join(data_folder, folder)
    for f in os.listdir(sub_folder):
      if not f.startswith("."): 
        try:
          # img = cv2.imread(data_folder + "/" + f, cv2.IMREAD_COLOR)
          img = load_img(sub_folder + "/" + f)  # this is a PIL image
          # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
          # img_resize = cv2.resize(img,(128,128))
          # print (img_resize.shape)
          
        except:
          print ("Exception")
          continue
        
        img = img.resize((224, 224), Image.ANTIALIAS)
        # x = img_to_array(img)  
        # print (x)
        # x = x.resize((3,224,224))
        # print (x)
        # x = rgb2gray(x)
        # print (x.shape)
        # np.asarray(vector).reshape(shape)
        x = np.asarray(img).reshape(3,224, 224)
        # print (x.shape)

        # img.thumbnail((480, 640))
        # x = x.resize(1, 224, 224)
        # print (x)
        # break
        dataset_train[i] = x
        i += 1
        if i % 250 == 0:
            print("%d images to array" % i)

# img_data = np.array(dataset_train)
# img_data = img_data.astype('float32')
# img_data /=255
print(dataset_train.shape)
print("All images to array!")


FileNotFoundError: ignored

In [0]:
dataset_train[1]
np.save('/content/drive/My Drive/NN-ProjectC/Project_C1/Training/data_labeled_color.npy', dataset_train)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from sklearn.model_selection import train_test_split
import numpy as np
import os

dataset1 = np.load('/content/drive/My Drive/NN-ProjectC/Project_C1/Training/data_labeled_color.npy', allow_pickle=True)
dataset1 = np.reshape(dataset1, (len(dataset1), 224, 224, 3))
# print (dataset1.shape)
# dataset1 = dataset1.reshape(-1, 281186304)
# dataset1 /=255
# print (dataset1[1317])
# Y = np.load('/content/drive/My Drive/NN-ProjectC/Project_C1/Training/data_labeled_labels.npy', allow_pickle=True)
# print (Y.shape)
# dataset1 = dataset1.reshape(1868, 150528)
# dataset1[0]
train_x, val_x = train_test_split(dataset1, test_size=0.3)
val_x.shape

(561, 224, 224, 3)

In [0]:
# train_x = train_x.reshape(-1, 150528)
# val_x = val_x.reshape(-1, 150528)
# train_x = train_x.reshape(1307, 224, 224, 3)
# val_x = val_x.reshape(561, 224, 224, 3)
# train_x.shape

(1307, 224, 224, 3)

In [0]:
import pandas as pd
df = pd.DataFrame(train_x)
df

In [0]:
import keras

import keras.backend as K

from time import time
from sklearn.cluster import KMeans
from keras import callbacks
from keras.models import Model
from keras.optimizers import SGD
from keras.layers import Dense, Input, BatchNormalization, Reshape, Flatten
from keras.initializers import VarianceScaling
from keras.engine.topology import Layer, InputSpec

from sklearn.metrics import accuracy_score, normalized_mutual_info_score
img_size = 224

total_pixels = img_size * img_size * 3
translator_factor = 2
translator_layer_size = int(total_pixels/translator_factor)
middle_factor = 2
middle_layer_size = int(translator_layer_size/middle_factor)

# this is our input placeholder
# input_img = Input(shape=(150528,))
inputs = Input(shape=(img_size,img_size,3))
x = Flatten(name = 'flattened_cat')(inputs)

x = BatchNormalization()(x)
x = Dense(translator_layer_size, activation='relu', name='encoder')(x)
x = Dense(middle_layer_size, activation='relu', name='middle_layer')(x)
x = BatchNormalization()(x)
x = Dense(translator_layer_size, activation='relu', name='decoder')(x)

outputs = Dense(total_pixels, activation='sigmoid', name='reconstructed_cat')(x)
outputs = Reshape((img_size,img_size,3))(outputs)

model = Model(inputs=inputs, outputs=outputs)
model.summary()
# # "encoded" is the encoded representation of the input
# encoded = Dense(5000, activation='relu')(input_img)
# encoded = Dense(2000, activation='relu')(encoded)
# encoded = Dense(500, activation='relu')(encoded)
# encoded = Dense(500, activation='relu')(encoded)
# encoded = Dense(10, activation='sigmoid')(encoded)

# # "decoded" is the lossy reconstruction of the input
# decoded = Dense(500, activation='relu')(encoded)
# decoded = Dense(500, activation='relu')(decoded)
# decoded = Dense(2000, activation='relu')(decoded)
# decoded = Dense(5000, activation='relu')(decoded)
# decoded = Dense(150528)(decoded)

# # this model maps an input to its reconstruction
# autoencoder = Model(input_img, decoded)
# autoencoder.summary()

Using TensorFlow backend.






Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
flattened_cat (Flatten)      (None, 150528)            0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 150528)            602112    
_________________________________________________________________
encoder (Dense)              (None, 75264)             1132941465
_________________________________________________________________
middle_layer (Dense)         (None, 37632)             2832372480
_________________________________________________________________
batch_normalization_2 (Batch (None, 37632)             150528    
_________________________________________________________________
decoder (Dense)              (None, 75264)             

In [0]:
customAdam = keras.optimizers.Adam(lr=0.001) 
model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="mean_squared_error",
              # List of metrics to monitor
              metrics=["mean_squared_error"])

In [0]:
print('# Fit model on training data')

history = model.fit(train_x, 
                    train_x, #we pass it the same input data as desired output
                    
                    #If the model is taking forever to train, make this bigger
                    #If it is taking forever to load for the first epoch, make this smaller
                    batch_size=256, 
                    
                    epochs=10,
                    # We pass it validation data to
                    # monitor loss and metrics
                    # at the end of each epoch
                    validation_data=(val_x, val_x))

# Fit model on training data



Train on 1307 samples, validate on 561 samples
Epoch 1/10







In [0]:
#  this model maps an input to its encoded representation
encoder = Model(input_img, encoded)
autoencoder.compile(optimizer='adam', loss='mse')
estop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
train_history = autoencoder.fit(train_x, train_x, epochs=500, batch_size=4, validation_data=(val_x, val_x), callbacks=[estop])






Train on 1307 samples, validate on 561 samples
Epoch 1/500





